
[bearing Dataset](https://zhuanlan.zhihu.com/p/88938034)

### CWRU Dataset
Data was collected for normal bearings, single-point drive end and fan end defects.  
Data was collected at 12,000 samples/second and at 48,000 samples/second for drive end bearing experiments.  
All fan end bearing data was collected at 12,000 samples/second.  

- DE - drive end accelerometer data (驅動端加速度數據)
- FE - fan end accelerometer data (風扇端加速度數據)
- BA - base accelerometer data (電機支撐基本平板加速度數據)
- time - time series data (時間序列數據)
- RPM- rpm during testing (轉速)
- [理論知識](https://zhuanlan.zhihu.com/p/40852333)
- [滚动轴承故障特征频率的计算](http://blog.sina.com.cn/s/blog_574d08530100p3pr.html)

In [2]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
import tensorflow as tf
import keras

%matplotlib inline

Using TensorFlow backend.


In [ ]:
# X100_DE = rawdata['X100_DE_time']
# X100_FE = rawdata['X100_FE_time']
# X100_RPM = rawdata['X100RPM']

In [2]:
batch_size = 128
epochs = 12
num_classes = 10
length = 2048
BatchNorm = True # 是否批量归一化
number = 1000 # 每类样本的数量
normal = True # 是否标准化
rate = [0.7,0.2,0.1] # 测试集验证集划分比例
date=time.strftime("%Y%m%d", time.localtime())
mark=time.strftime("%Y%m%d_%H%M", time.localtime())

In [5]:
import preprocess
path = '/home/cslab/Documents/CWRU/12DriveEndFault/1797/12DriveEndFault/1797/'
x_train, y_train, x_valid, y_valid, x_test, y_test = preprocess.prepro(d_path=path,
                                                                length=length,
                                                                number=number,
                                                                normal=False,
                                                                rate=rate,
                                                                enc=True,
                                                                enc_step=28)

In [6]:
x_train, x_valid, x_test = x_train[:,:,np.newaxis], x_valid[:,:,np.newaxis], x_test[:,:,np.newaxis]

In [7]:
input_shape =x_train.shape[1:]

In [8]:
print('train shape:', x_train.shape)
print(x_train.shape[0], 'train 樣本個數')
print('valid shape', x_valid.shape)
print(x_valid.shape[0], 'valid 樣本個數')
print('test shape', x_test.shape)
print(x_test.shape[0], 'test 樣本個數')

train shape: (10500, 2048, 1)
10500 train 樣本個數
valid shape (3000, 2048, 1)
3000 valid 樣本個數
test shape (1500, 2048, 1)
1500 test 樣本個數


In [12]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Flatten

In [15]:
model = Sequential()

model.add(LSTM(32, activation='tanh', 
               recurrent_activation='hard_sigmoid',
               kernel_initializer='glorot_uniform', 
               recurrent_initializer='orthogonal', 
               bias_initializer='zeros', 
               return_sequences=True))

          
          
model.add(Flatten())

# 增加输出层，共num_classes个单元，激活函数为softmax
model.add(Dense(units=num_classes, activation='softmax'))

# 编译模型 评价函数和损失函数相似，不过评价函数的结果不会用于训练过程中
model.compile(optimizer='Adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# 开始模型训练
model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_valid, y_valid), shuffle=True)


ValueError: Error when checking target: expected dense_2 to have shape (10,) but got array with shape (15,)

In [ ]:
# 评估模型
score = model.evaluate(x=x_test, y=y_test, verbose=0)
print("测试集上的损失率：", score[0])
print("测试集上的准确率：", score[1])
# plot_model(model=model, to_file='images/lstm-diagnosis.png', show_shapes=True)

### Python 讀取.mat文件

In [ ]:
X = loadmat(r"filename")
data_all = X['data'] # data為結構體的名稱
print (data_all.shape)

### 結果adat_all的shape為(1, 1)
### 解決方法
[scipy.io.loadmat](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html#scipy.io.loadmat)

In [ ]:
data_all = X['data']
data = data_all[0,0]['SYSLoad']
print(data.shape)

### 查看 Raw Data

In [3]:
X = loadmat("/home/cslab/Documents/CWRU/12DriveEndFault/1797/12DriveEndFault/1797/0.007-Ball.mat")

In [8]:
X.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X118_DE_time', 'X118_FE_time', 'X118_BA_time', 'X118RPM'])

In [11]:
X

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon Jan 31 13:55:05 2000',
 '__version__': '1.0',
 '__globals__': [],
 'X118_DE_time': array([[-0.0027614 ],
        [-0.09632403],
        [ 0.11370459],
        ...,
        [-0.07829373],
        [-0.14911545],
        [ 0.02111657]]),
 'X118_FE_time': array([[-0.24716182],
        [ 0.14279091],
        [ 0.00328727],
        ...,
        [ 0.01643636],
        [-0.03842   ],
        [-0.16806182]]),
 'X118_BA_time': array([[ 0.01553163],
        [ 0.01693994],
        [-0.03645507],
        ...,
        [ 0.03178754],
        [-0.03243134],
        [-0.0668343 ]]),
 'X118RPM': array([[1796]], dtype=uint16)}

In [12]:
data = X['X118_DE_time'][0, :]

In [13]:
data

array([-0.0027614])